## Doing a EDA about texi predictions, starting out with cleaning the data and understanding it

In [4]:
import pandas as pd
import numpy as np



In [5]:
df = pd.read_csv("../src/taxipred/data/taxi_trip_pricing.csv")

In [6]:
df.shape

(1000, 11)

In [7]:
df.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [11]:
df["Trip_Price"].value_counts()

Trip_Price
36.2624    1
52.9032    1
36.4698    1
15.6180    1
60.2028    1
          ..
34.4049    1
62.1295    1
33.1236    1
61.2090    1
45.4437    1
Name: count, Length: 951, dtype: int64

In [13]:
df["Trip_Price"].mean()

np.float64(56.87477332571041)

In [14]:
df["Per_Km_Rate"].mean()

np.float64(1.2333157894736844)

In [15]:
df["Time_of_Day"].value_counts()


Time_of_Day
Afternoon    371
Morning      283
Evening      203
Night         93
Name: count, dtype: int64

In [16]:
df_night = df[df["Time_of_Day"] == "Night"]
df_night.shape

(93, 11)

In [18]:
df_night["Trip_Price"].value_counts()

Trip_Price
88.132800     1
27.441500     1
69.626800     1
201.869509    1
39.236600     1
             ..
54.368400     1
93.509600     1
47.625200     1
59.607500     1
62.129500     1
Name: count, Length: 89, dtype: int64

In [19]:
df["Base_Fare"].mean()

np.float64(3.502989473684211)

In [20]:
df["Base_Fare"].value_counts()

Base_Fare
3.94    10
3.88     8
2.32     8
2.80     8
2.38     7
        ..
2.89     1
2.78     1
2.37     1
3.04     1
2.67     1
Name: count, Length: 290, dtype: int64

In [21]:
df.isnull().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

In [1]:
df_group_sum = (
    df.groupby("Day_of_Week", as_index=False) 
    .agg(Total_Km=("Trip_Distance_km", "sum"),
    Total_Minutes=("Trip_Duration_Minutes", "sum"),
    Total_Price=("Trip_Price", "sum"))
    .sort_values(["Day_of_Week"])
)

df_group_sum.shape

NameError: name 'df' is not defined